## 内容一：利用CNN对pytorch内置数据集MNIST进行手写识别分类


###  实验步骤

#### 实验前导

In [1]:
# 导入必要的包
import numpy as np
import torch
import torchvision 
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.utils.data as tud

#### 准备数据

In [2]:
# 定义dataset
mnist_train_data= datasets.MNIST("./mnist_data",train=True,download=True,transform=transforms.ToTensor())
# 定义dataloader
mnist_train_dataloader=tud.DataLoader(mnist_train_data,batch_size=64,shuffle=True)#DataLoader类

In [3]:
mnist_test_data= datasets.MNIST("./mnist_data",train=False,download=True,transform=transforms.ToTensor())
mnist_test_dataloader=tud.DataLoader(mnist_test_data,batch_size=64,shuffle=True)#DataLoader类

In [4]:
mnist_train_data_numpy=datasets.MNIST("./mnist_data",train=True,download=False)
# 查看数据
print(len(mnist_test_dataloader.dataset))
print("train_data:", mnist_test_data.test_data.size())
print("train_data:", mnist_train_data.train_data.size())# datdaset类中要分mnist_train_data.train_data.size和mnist_train_data.train_labels
print("train_labels:", mnist_train_data.train_labels.size())
# 查看原本的数据集形式及标签
print(mnist_train_data_numpy[0])
mnist_train_data_numpy[0][0]# 查看标签

10000
train_data: torch.Size([10000, 28, 28])
train_data: torch.Size([60000, 28, 28])
train_labels: torch.Size([60000])
(<PIL.Image.Image image mode=L size=28x28 at 0x19B7BE2DE48>, 5)


C:\Users\28038\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\28038\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
C:\Users\28038\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:45: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


#### 配置网络

In [5]:
# 定义简单的卷积神经网络 输入的size:(1*28*28)
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()#复制并使用Net的父类的初始化方法，即先运行nn.Module的初始化函数
        self.conv1=nn.Conv2d(1,20,5,1)# 输入为图像（1个频道，即灰度图）,输出为 20张特征图（通道数）, 卷积核为5x5正方形，【（28-5）/1】+1=24
        self.conv2=nn.Conv2d(20,50,5,1)# 50个5*5的卷积核，输出为 50张特征图（通道数）步长为1   【（24-5）/1】+1=20
        self.fc1=nn.Linear(4*4*50,500)# 将50*4*4个节点通过线性函数连接到500个节点上
        self.fc2=nn.Linear(500,10)# 将500个节点通过线性函数连接到10个节点上，也就是分10类
# 定义向前传播
    def forward(self,x):
        x=F.relu(self.conv1(x))# 结果20*24*24
        x=F.max_pool2d(x,2,2)# 使用2x2的窗口进行最大池化Max pooling，然后更新到x，结果20*12*12   12-5+1=8 
        x=F.relu(self.conv2(x))# 结果50*8*8
        x=F.max_pool2d(x,2,2)# 结果50*4*4
        x=x.view(-1,4*4*50)#view函数将张量x变形成一维的向量形式，总特征数并不改变，为接下来的全连接作准备
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return F.log_softmax(x,dim=1)


In [6]:
### 实例化模型
mnist_model=Net()
mnist_model = mnist_model.cuda()
# 定义损失函数
Loss_fn= nn.CrossEntropyLoss()
# 定义优化策略
optimizer=optim.SGD(mnist_model.parameters(),lr=0.01,momentum=0.5)   # Net.parameters()要和上面的Net一致
### 易错使用Net.parametres()一定要进行实例化，就是先对模型实例化

#### 定义训练和测试函数

In [7]:
# 定义训练网络函数
def Train(model,train_dataloader,Loss_fn,optimizer,epoch):
    model.train()#使用BatchNormalization 和 Dropout 
    for idx,(img,label) in enumerate(train_dataloader):
        img,label=img.cuda(),label.cuda()
        output=model(img)# 
        loss=Loss_fn(output,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if idx%100==0:
            print("train_epoch:{}，iteration：{}，loss:{:.5f},".format(epoch,idx,loss.item()))# :.5f保留五位
    return model# 方便取最优模型
## 定义测试网络的函数        
def Test(model,test_dataloader,Loss_fn):
    model.eval() #不使用BatchNormalization 和 Dropout 
    total_loss=0. #0后面有个点，浮点型，才可以让最后结果不为零
    total_accnum=0.
    with torch.no_grad():# 不需要反向求导进行优化，所以不用梯度
        for idx,(img,label) in enumerate(test_dataloader):
            img,label=img.cuda(),label.cuda()
            output=model(img)# output 64*10
            loss=Loss_fn(output,label)
            pred= output.argmax(dim=1)#torch.argmax()返回指定维度最大值的序号 特别的在dim=0表示二维中的列，dim=1在二维矩阵中表示行
            # 测试需要计算准确率
            total_loss += loss
            total_accnum += pred.eq(label).sum()
            # total_accnum += (pred==label).sum()
            
    total_loss/=len(test_dataloader.dataset)
    acc=total_accnum/len(test_dataloader.dataset)
    print("test_loss:{:.5f}，准确率:{:.5f}".format(total_loss,acc))
    return acc#方便取最优模型


- debug技巧

    - 十类，每一类概率都是0.1，根据loss的初始化，初始值应该是log10，所以可以判断最开始的loss是2.3是否合适
    - 如果反复运行训练两个epochs，loss会不断优化，第一次迭代的loss就在上一次结果的loss优化，准确率越来越高

In [9]:
np.log(10)

2.302585092994046

####  模型保存和加载及打印参数

In [10]:
# torch.save(model.state_dict(),"mnist_cnn.pth") # 只保存神经网络的模型参数   
#t保存最优模型
num_epochs= 2
best_valid_acc = 0.
for epoch in range(num_epochs):
    Train(mnist_model,mnist_train_dataloader,Loss_fn,optimizer,epoch)
    acc = Test(mnist_model,mnist_test_dataloader,Loss_fn)
    if acc > best_valid_acc:
        best_valid_acc = acc
        torch.save(mnist_model.state_dict(),"best_mnist_cnn.pth")

train_epoch:0，iteration：0，loss:2.30064,
train_epoch:0，iteration：100，loss:2.21395,
train_epoch:0，iteration：200，loss:0.85474,
train_epoch:0，iteration：300，loss:0.60452,
train_epoch:0，iteration：400，loss:0.39538,
train_epoch:0，iteration：500，loss:0.21170,
train_epoch:0，iteration：600，loss:0.50295,
train_epoch:0，iteration：700，loss:0.13989,
train_epoch:0，iteration：800，loss:0.51564,
train_epoch:0，iteration：900，loss:0.22044,
test_loss:0.00276，准确率:0.94810
train_epoch:1，iteration：0，loss:0.16161,
train_epoch:1，iteration：100，loss:0.14594,
train_epoch:1，iteration：200，loss:0.20588,
train_epoch:1，iteration：300，loss:0.07314,
train_epoch:1，iteration：400，loss:0.11452,
train_epoch:1，iteration：500，loss:0.07849,
train_epoch:1，iteration：600，loss:0.12429,
train_epoch:1，iteration：700，loss:0.21019,
train_epoch:1，iteration：800，loss:0.09245,
train_epoch:1，iteration：900，loss:0.14659,
test_loss:0.00155，准确率:0.96850


In [11]:
# Print mnist_model's state_dict
print("Model's state_dict:")
for param_tensor in mnist_model.state_dict():
    print(param_tensor, "\t", mnist_model.state_dict()[param_tensor].size())


# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.weight 	 torch.Size([20, 1, 5, 5])
conv1.bias 	 torch.Size([20])
conv2.weight 	 torch.Size([50, 20, 5, 5])
conv2.bias 	 torch.Size([50])
fc1.weight 	 torch.Size([500, 800])
fc1.bias 	 torch.Size([500])
fc2.weight 	 torch.Size([10, 500])
fc2.bias 	 torch.Size([10])
Optimizer's state_dict:
state 	 {1767310573304: {'momentum_buffer': tensor([[[[-8.2635e-03, -1.4580e-02, -1.7740e-02, -1.9181e-02, -2.1253e-02],
          [-7.0006e-03, -1.2943e-02, -1.5249e-02, -1.9282e-02, -2.1084e-02],
          [-4.2616e-03, -8.7391e-03, -9.9263e-03, -1.1852e-02, -1.6189e-02],
          [-4.3360e-03, -3.4405e-03, -4.4456e-03, -7.9326e-03, -1.7687e-02],
          [ 7.3884e-04,  1.0026e-03, -4.0219e-03, -1.2944e-02, -2.0509e-02]]],


        [[[ 4.0375e-03,  3.0989e-03, -1.8878e-04, -2.6025e-03, -1.7005e-03],
          [ 2.9432e-03,  1.5412e-03, -2.1823e-03, -4.0925e-03, -5.3510e-03],
          [ 3.4433e-03,  1.3687e-03, -5.4327e-03, -1.0017e-02, -9.5813e-03],
          [ 1.0118e-0

In [12]:
test_model= Net()## 前面实例化类命名的时候，要避免重复Net=Net()这种，否则后面再实例化时会出错，不认得类Net
test_model.load_state_dict(torch.load("mnist_cnn.pth"))
test_model= test_model.cuda()  #要在这一步使用cuda,否则模型在GPU，参数就在cpu
Test(test_model,mnist_test_dataloader,Loss_fn)

test_loss:0.00166，准确率:0.96820


tensor(0.9682, device='cuda:0')

### For FashionMNIST

In [13]:
# 数据准备train
transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize(mean=(0.2860402,),std=(0.3530239,))
                   ])
fashion_train= datasets.FashionMNIST("./fashion_mnist_data",train=True,transform=transform)
fashion_train_dataloader=tud.DataLoader(fashion_train,batch_size=32,shuffle=True)#DataLoader类
# 数据准备test
fashion_test= datasets.FashionMNIST("./fashion_mnist_data",train=False,transform=transform)
fashion_test_dataloader=tud.DataLoader(fashion_test,batch_size=32,shuffle=True)#DataLoader类

In [14]:
fashion_train
fashion_train[0][0] .size()#数组，第一维是tensor，第二维标签,可以进行标准化，计算如下

torch.Size([1, 28, 28])

In [15]:
fashion_train_dataloader

In [16]:
fashion_model = Net()
fashion_model = fashion_model.cuda()# 定义Train和Test时就把输入数据放cuda上，所以这里模型也要做cuda
loss_fn = nn.CrossEntropyLoss(reduction='mean')
lr = 0.01;momentum = 0.5
optimizer = optim.SGD(fashion_model.parameters(),lr=lr,momentum=momentum)

In [17]:
num_epochs = 2
for epoch in range(num_epochs):
    model = Train(fashion_model,fashion_train_dataloader,loss_fn,optimizer,epoch)
    acc = Test(fashion_model,fashion_test_dataloader,loss_fn)

train_epoch:0，iteration：0，loss:2.30442,
train_epoch:0，iteration：100，loss:0.78413,
train_epoch:0，iteration：200，loss:0.79574,
train_epoch:0，iteration：300，loss:0.98147,
train_epoch:0，iteration：400，loss:0.70490,
train_epoch:0，iteration：500，loss:0.49413,
train_epoch:0，iteration：600，loss:0.60802,
train_epoch:0，iteration：700，loss:0.60331,
train_epoch:0，iteration：800，loss:0.76761,
train_epoch:0，iteration：900，loss:0.67921,
train_epoch:0，iteration：1000，loss:0.73171,
train_epoch:0，iteration：1100，loss:0.29316,
train_epoch:0，iteration：1200，loss:0.47057,
train_epoch:0，iteration：1300，loss:0.46732,
train_epoch:0，iteration：1400，loss:0.57553,
train_epoch:0，iteration：1500，loss:0.41564,
train_epoch:0，iteration：1600，loss:0.39209,
train_epoch:0，iteration：1700，loss:0.44661,
train_epoch:0，iteration：1800，loss:0.49080,
test_loss:0.01394，准确率:0.83670
train_epoch:1，iteration：0，loss:0.35871,
train_epoch:1，iteration：100，loss:0.29302,
train_epoch:1，iteration：200，loss:0.46073,
train_epoch:1，iteration：300，loss:0.37371,

In [18]:
torch.save(fashion_model.state_dict(),"fashion_mnist_cnn.pth")
print("fashion_model's state_dict:")
for param_tensor in fashion_model.state_dict():
    print(param_tensor, "\t", fashion_model.state_dict()[param_tensor].size())

fashion_model's state_dict:
conv1.weight 	 torch.Size([20, 1, 5, 5])
conv1.bias 	 torch.Size([20])
conv2.weight 	 torch.Size([50, 20, 5, 5])
conv2.bias 	 torch.Size([50])
fc1.weight 	 torch.Size([500, 800])
fc1.bias 	 torch.Size([500])
fc2.weight 	 torch.Size([10, 500])
fc2.bias 	 torch.Size([10])


- 计算均值和标准差

In [24]:
y = 0
for a in fashion_train:
     y = a[0].numpy().sum().mean()
y / len(fashion_train)  #0.2860406029704958
#sum = 0
#for a in fashion_train:
#    sum += a[0].numpy().std()
#sum / len(fashion_train) #0.3202489306078603

-0.007498455810546875